In [ ]:
import panel as pn
pn.extension()

A widget to display a list of `ChatEntry` objects and interact with them.

This widget provides backend methods to:
- Send (append) messages to the chat log.
- Stream tokens to the latest `ChatEntry` in the chat log.
- Execute callbacks when a user sends a message.
- Undo a number of sent `ChatEntry` objects.
- Clear the chat log of all `ChatEntry` objects.

See [`ChatInterface`](ChatInterface.ipynb) for the frontend counterpart of `ChatFeed`, featuring interactive input widgets and convenient action buttons that leverage the backend methods of `ChatFeed`.

![Chat Design Specification](../../assets/ChatDesignSpecification.png)

#### Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

##### Core

* **`value`** (List[ChatEntry]): The entries added to the chat feed.
* **`header`** (str | Widget): The header of the chat feed. Can be a string, pane, or widget.
* **`callback`** (Callable): Callback to execute when a user sends a message or when `respond` is called. The signature must include the previous message value `contents`, the previous `user` name, and the component `instance`.
* **`callback_user`** (str): The default user name to use for the entry provided by the callback.
* **`callback_avatar`** (str | BinaryIO): The default avatar to use for the entry provided by the callback.
* **`placeholder`** (Any): Placeholder to display while the callback is running. If not set, defaults to a LoadingSpinner.
* **`placeholder_text`** (str): If the placeholder is the default LoadingSpinner, the text to display next to it.
* **`placeholder_threshold`** (float): Minimum duration in seconds of buffering before displaying the placeholder. If 0, the placeholder will be disabled.
* **`auto_scroll_limit`** (int): Maximum pixel distance from the latest object in the Column to activate automatic scrolling upon update. Setting to 0 disables auto-scrolling.
* **`scroll_button_threshold`** (int): Minimum pixel distance from the latest object in the Column to display the scroll button. Setting to 0 disables the scroll button.
* **`view_latest`** (bool): Whether to scroll to the latest object on initialization. If not enabled, the view will be on the first object.

##### Styling

* **`card_params`** (Dict): Parameters to pass to Card, such as `header`, `header_background`, `header_color`, etc.
* **`entry_params`** (Dict): Parameters to pass to each ChatEntry, such as `reaction_icons`, `timestamp_format`, `show_avatar`, `show_user`, and `show_timestamp`.

___

In [ ]:
chat_feed = pn.widgets.ChatFeed()
chat_feed.send("Hello world!", user="Bot", avatar="🤖")

Besides messages of `str` type, the `send` method can also accept `dict`s containing the key `value` and `ChatEntry` objects.

In [ ]:
chat_feed = pn.widgets.ChatFeed()
chat_feed.send({"value": "Welcome!", "user": "Bot", "avatar": "B"})

Note if you provide both the user/avatar in the `dict` and keyword argument, the keyword argument takes precedence.

In [ ]:
chat_feed = pn.widgets.ChatFeed()
chat_feed.send({"value": "Overtaken!", "user": "Bot"}, user="MegaBot")

The primary method for `ChatFeed` is `send`, however it's possible to pre-populate the `ChatFeed` with `ChatEntry` objects.

In [ ]:
from asyncio import sleep

from panel.widgets.chat import ChatEntry, ChatFeed

pn.extension()

async def get_response(contents, user, instance):
    await sleep(1)
    return {
        "Philipp": None,  # Ignore Philipp :-)
        "Marc": "It is 2",
        "Andrew": "It is 4",
    }.get(user, "I don't know")

ASSISTANT_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/6/60/Icons8_flat_assistant.svg"

chat_feed = ChatFeed(
    value=[ChatEntry(user="Assistant", value="Hello World", avatar=ASSISTANT_AVATAR)],
    callback=get_response,
    height=500,
    callback_avatar=ASSISTANT_AVATAR,
)

marc_button = pn.widgets.Button(
    name="Marc",
    on_click=lambda event: chat_feed.send(
        user="Marc", value="What is the square root of 4", avatar="🚴"
    ),
    align="center",
)
andrew_button = pn.widgets.Button(
    name="Andrew",
    on_click=lambda event: chat_feed.send(
        user="Andrew", value="What is the square root of 4 squared?", avatar="🧑"
    ),
    align="center",
)
philipp_button = pn.widgets.Button(
    name="Philipp",
    on_click=lambda event: chat_feed.send(
        user="Philipp", value="What is 2+2", avatar="🏃"
    ),
    align="center",
)

pn.Column(
    chat_feed,
    pn.layout.Divider(),
    pn.Row("Click to ask a question", marc_button, andrew_button, philipp_button),
).servable()


Note how we use emojis and images (via urls) as avatars in the `ChatFeed`.

We chose to use an *async* callback in the example. We recommend using *async* callbacks and *async* functions from `openai` and other AI frameworks whenever possible for optimal performance.